In [1]:
import ast, numpy, random, requests, json, operator
import pylab as plt
from pyelasticsearch.client import ElasticSearch

In [42]:
data_file = open('data/train.json', 'r')
train_recipes = ast.literal_eval(data_file.read())
data_file = open('data/test.json', 'r')
test_recipes = ast.literal_eval(data_file.read())

In [44]:
nb_train_recipes = len(train_recipes)
print nb_train_recipes

39774


In [45]:
def get_cuisine_distribution(recipes):
    cuisine_distribution = {}
    for recipe in recipes:
        if recipe['cuisine'] in cuisine_distribution:
            cuisine_distribution[recipe['cuisine']] += 1
        else:
            cuisine_distribution[recipe['cuisine']] = 1

    print cuisine_distribution
    for cuisine in cuisine_distribution:
        cuisine_distribution[cuisine] =  float(cuisine_distribution[cuisine])/float(len(recipes))
    return cuisine_distribution

CUISINE_DISTRIBUTION = get_cuisine_distribution(train_recipes)

{'irish': 667, 'mexican': 6438, 'chinese': 2673, 'filipino': 755, 'vietnamese': 825, 'moroccan': 821, 'brazilian': 467, 'japanese': 1423, 'british': 804, 'greek': 1175, 'indian': 3003, 'jamaican': 526, 'french': 2646, 'spanish': 989, 'russian': 489, 'cajun_creole': 1546, 'thai': 1539, 'southern_us': 4320, 'korean': 830, 'italian': 7838}


In [28]:
plt.bar(range(len(cuisine_distribution)), cuisine_distribution.values())
plt.xticks(range(len(cuisine_distribution)), cuisine_distribution.keys())
plt.show()

NameError: name 'cuisine_distribution' is not defined

In [5]:
random_recipes_test = []
for generated_test_nb in range(0, nb_train_recipes/4):
    recipe = random.choice(train_recipes)
    while recipe in random_recipes_test:
        recipe = random.choice(train_recipes)
    random_recipes_test.append(recipe)

random_recipes_train = []
for recipe in train_recipes:
    if recipe not in random_recipes_test:
        random_recipes_train.append(recipe)

In [225]:
distribution_random_test = get_cuisine_distribution(random_recipes_test)
print distribution_random_test
distribution_random_train = get_cuisine_distribution(random_recipes_train)
print distribution_random_train

{'british': 142, 'irish': 125, 'mexican': 1214, 'filipino': 145, 'chinese': 511, 'vietnamese': 150, 'brazilian': 89, 'japanese': 255, 'french': 529, 'greek': 217, 'indian': 520, 'cajun_creole': 287, 'moroccan': 158, 'spanish': 178, 'russian': 114, 'jamaican': 92, 'thai': 316, 'southern_us': 777, 'korean': 151, 'italian': 1487}
{'british': 0.019042510392919404, 'irish': 0.0167627732332037, 'mexican': 0.16280005364087435, 'filipino': 0.019444816950516294, 'chinese': 0.06852621697733673, 'vietnamese': 0.020115327879844443, 'brazilian': 0.011935094542041036, 'japanese': 0.03419605739573555, 'french': 0.07094005632291807, 'greek': 0.029100174332841625, 'indian': 0.0697331366501274, 'cajun_creole': 0.0384873273434357, 'moroccan': 0.021188145366769478, 'spanish': 0.02387018908408207, 'russian': 0.015287649188681775, 'jamaican': 0.012337401099637924, 'thai': 0.042376290733538956, 'southern_us': 0.1041973984175942, 'korean': 0.02024943006571007, 'italian': 0.19940995038219123}
{'irish': 389, 'm

In [46]:
random_recipes_train = list_to_string_ingredients(random_recipes_train)
random_recipes_test = list_to_string_ingredients(random_recipes_test)

In [30]:
print random_recipes_train[0]

{'cuisine': 'italian', 'id': 18680, 'ingredients': 'powdered sugar, cannoli shells, chopped pecans, granulated sugar, heavy cream, mascarpone, butter, pears, lemon zest, sauce'}


In [7]:
def list_to_string_ingredients(recipes):
    for recipe in recipes:
        recipe['ingredients'] = ', '.join(recipe['ingredients'])
    return recipes

def index_recipes_in_ES(index, recipes):
    es = ElasticSearch()
    map_id_esid = {}
    for recipe in recipes:
        res = es.index(index=index, doc_type='recipe', id=recipe['id'], doc=recipe)
        if res['created'] == False:
            print res

In [47]:
index_recipes_in_ES('train-recipe', random_recipes_train)
index_recipes_in_ES('test-recipe', random_recipes_test)

In [11]:
def build_mlt(nb, doc_id):
    mlt = {}
    mlt["from"] = 0
    mlt["size"] = nb
    mlt["query"] = {}
    mlt["query"]["more_like_this"] = {}
    mlt["query"]["more_like_this"]["fields"] = ["ingredients"]
    mlt["query"]["more_like_this"]["like"] = [{"_index" : "test-recipe","_type" : "recipe","_id" : doc_id}]
    mlt["query"]["more_like_this"]["min_term_freq"] = 1
    mlt["query"]["more_like_this"]["max_query_terms"] = 50
    mlt["query"]["more_like_this"]["minimum_should_match"] = "25%"
    return mlt

def extract_from_json(json):
    hits = json['hits']['hits']
    recipes = [hit['_source'] for hit in hits]
    return recipes

def get_similar(nb, doc_id):
    mlt = build_mlt(nb, doc_id)
    response = requests.post("http://localhost:9200/train-recipe/recipe/_search", data=json.dumps(mlt))
    similar_recipes = extract_from_json(json.loads(response.text))
    return similar_recipes

def get_document_by_id(index, doc_id):
    es = ElasticSearch()
    response = es.get(index=index, doc_type='recipe', id=doc_id)
    return response['_source']


In [12]:
def get_cuisine_from_recipe_id(recipe_index, recipe_id):
    for recipe in recipe_index:
        if recipe['id'] == recipe_id:
            return recipe['cuisine']

def get_result_test(results, random_recipes_test):
    succes = []
    fail = []
    for result in results:
        real_cuisine = get_document_by_id('test-recipe', result[1])
        predict_cuisine = result[0]
        if real_cuisine['cuisine'] == predict_cuisine:
            succes.append(real_cuisine)
        else:
            fail.append(real_cuisine)
    succes = Result_percentage(succes, float(len(random_recipes_test))) 
    fail = Result_percentage(fail, float(len(random_recipes_test))) 
    return succes, fail

class Result_percentage:
    def __init__(self, recipes, length_test):
        distribution = {}
        for recipe in recipes:
            cuisine = recipe['cuisine']
            if cuisine not in distribution:
                distribution[cuisine] = 1
            else:
                distribution[cuisine] += 1
        for cuisine in distribution:
            distribution[cuisine] = distribution[cuisine] * 100 / float(len(recipes))
        self.distribution = distribution
        self.percentage = len(recipes) / length_test * 100
    

In [13]:
def predict(recipes_to_predict, predict_origin, mlt_nb=100):
    predictions = []
    for recipe in recipes_to_predict:
        similar_recipes = get_similar(mlt_nb, recipe['id'])
        prediction = predict_origin(similar_recipes)
        predictions.append((prediction, recipe['id']))
    return predictions

In [14]:
def max_country_origin(similar_recipes):
    distribution = {}
    for recipe in similar_recipes:
        if recipe['cuisine'] in distribution:
            distribution[recipe['cuisine']] += 1
        else:
            distribution[recipe['cuisine']] = 1
    prediction = max(distribution.iteritems(), key=operator.itemgetter(1))[0]
    return prediction

def diff_distribution(similar_recipes):
    distribution = {}
    for recipe in similar_recipes:
        if recipe['cuisine'] in distribution:
            distribution[recipe['cuisine']] += 1
        else:
            distribution[recipe['cuisine']] = 1
    for cuisine in distribution:
        distribution[cuisine] = distribution[cuisine] * 100 / len(similar_recipes) 
        distribution[cuisine] = distribution[cuisine] - CUISINE_DISTRIBUTION[cuisine]
    prediction = max(distribution.iteritems(), key=operator.itemgetter(1))[0]
    return prediction

In [48]:
results = predict(random_recipes_test, diff_distribution, 15)

In [18]:
score = get_result_test(results, random_recipes_test)

In [19]:
print score[0].distribution
print score[0].percentage
print "--------------"
print score[1].distribution
print score[1].percentage

{u'british': 0.8099352051835853, u'irish': 0.8999280057595392, u'mexican': 20.248380129589634, u'filipino': 1.1159107271418287, u'chinese': 7.919366450683945, u'vietnamese': 1.4398848092152627, u'brazilian': 0.7199424046076314, u'japanese': 2.4478041756659468, u'french': 4.499640028797696, u'greek': 2.5197984161267097, u'indian': 8.92728581713463, u'cajun_creole': 3.5817134629229663, u'moroccan': 1.8898488120950323, u'spanish': 1.2239020878329734, u'russian': 0.593952483801296, u'jamaican': 1.0259179265658747, u'thai': 3.7077033837293016, u'southern_us': 11.01511879049676, u'korean': 1.7458603311735061, u'italian': 23.668106551475883}
74.5071744669
--------------
{u'british': 5.576012624934245, u'irish': 4.0504997369805364, u'mexican': 6.470278800631247, u'filipino': 3.9978958442924775, u'chinese': 3.051025775907417, u'vietnamese': 3.840084166228301, u'brazilian': 3.051025775907417, u'japanese': 6.417674907943188, u'french': 12.256706996317728, u'greek': 4.83955812730142, u'indian': 3.

SANS ANALYZER, DISTRIBUTION, MLT15, 10MAXTF, 30%MINMATCH
==
72.1110328875
AVEC ANALYZER, DISTRIBUTION, MLT15, 10MAXTF, 30%MINMATCH
==
72.2719501157
AVEC ANALYZER, DISTRIBUTION, MLT15, 50MAXTF, 30%MINMATCH
==
74.4342753696
AVEC ANALYZER, DISTRIBUTION, MLT15, 50MAXTF, 25%MINMATCH
==
75.2313262706

In [73]:
distribution_based = {}
max_based = {}
for i in range(1, 201):
    results = predict(random_recipes_test, diff_distribution, i)
    score = get_result_test(results, random_recipes_test)
    distribution_based[i] = score
    print score[0], i
    results = predict(random_recipes_test, max_country_origin, i)
    score = get_result_test(results, random_recipes_test)
    max_based[i] = score
    print score[0], i

66.3180126722 1
66.3180126722 1
62.5465151363 2
63.743337021 2
67.6757517852 3
67.9070703007 3
69.5765865433 4
69.5564718898 4
70.8438097154 5
70.4214019914 5
71.1254148647 6
70.9242683295 6
71.5679372423 7
71.3064467465 7
71.8595997184 8
71.4472493211 8
71.8998290254 9
71.7288544705 9
71.9300010057 10
71.7288544705 10
72.0104596198 11
71.7187971437 11
71.919943679 12
71.6785678367 12
72.1110328875 13
71.7590264508 13
71.9300010057 14
71.5981092226 14
72.1110328875 15
71.8093130846 15
71.7690837775 16
71.6282812029 16
71.8294277381 17
71.6483958564 17
71.7791411043 18
71.5478225888 18
71.6685105099 19
71.6282812029 19
71.8394850649 20
71.748969124 20
71.5176506085 21
71.5277079352 21
71.8294277381 22
71.708739817 22
71.6182238761 23
71.6584531831 23
71.6383385296 24
71.5679372423 24
71.6282812029 25
71.5277079352 25
71.6081665493 26
71.4975359549 26
71.5478225888 27
71.3567333803 27
71.3869053606 28
71.2259881324 28
71.2259881324 29
71.1757014985 29
71.155586845 30
71.0449562506 30
71.

KeyboardInterrupt: 

In [49]:
def to_csv(predictions):
    f = csv.writer(open("results.csv", "wb+"))
    f.writerow(["id", "cuisine"])
    for prediction in predictions:
        f.writerow([prediction[1], prediction[0]])

to_csv(results)

Elasticsearch installation
--

In [ ]:
brew install elasticsearch
brew install kibana
kibana plugin --install elastic/sense
elasticsearch -d
kibana
localhost:9200 # Elasticsearch
http://localhost:5601/app/sense # Kibana